In [ ]:
from ipywidgets import HTML, VBox, HBox
import plotly.graph_objects as go
import pandas as pd
import plotly.express as px
from IPython.display import display


In [ ]:
kpi1 = HTML("<div style='padding:15px; background:#e0f7fa; border-radius:8px;'><h4>Total Receivable</h4><p>$6,621,280</p></div>")
kpi2 = HTML("<div style='padding:15px; background:#ffebee; border-radius:8px;'><h4>Total Payable</h4><p>$1,630,270</p></div>")
kpi3 = HTML("<div style='padding:15px; background:#f3e5f5; border-radius:8px;'><h4>Equity Ratio</h4><p>75.38%</p></div>")
kpi4 = HTML("<div style='padding:15px; background:#e8f5e9; border-radius:8px;'><h4>Debt Equity</h4><p>1.10%</p></div>")
kpi_row = HBox([kpi1, kpi2, kpi3, kpi4])


In [ ]:
df_bar = pd.DataFrame({
    'Client': ['Super1', 'Super2', 'Super3'],
    'Receivable': [200000, 150000, 180000],
    'Payable': [100000, 120000, 80000]
})
fig_bar = px.bar(df_bar, x='Client', y=['Receivable', 'Payable'], barmode='group')
bar_widget = go.FigureWidget(fig_bar)


In [ ]:
df_line = pd.DataFrame({
    'Month': pd.date_range("2025-01-01", periods=12, freq='M'),
    'Net WC': [10000, 12000, 11000, 13000, 12500, 14000, 15000, 14500, 16000, 17000, 16500, 18000],
    'Gross WC': [15000, 16000, 15500, 16500, 17000, 17500, 18000, 18500, 19000, 19500, 20000, 21000]
})
fig_line = px.line(df_line, x='Month', y=['Net WC', 'Gross WC'], markers=True)
line_widget = go.FigureWidget(fig_line)


In [ ]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import HTML
from IPython.display import display
import numpy as np
import re
import matplotlib.pyplot as plt

# ---------- Load Data ----------
df = pd.read_csv("naivas_products.csv")

# ---------- Clean Price Column ----------
df['Cleaned Price'] = (
    df['Price']
    .str.replace("KES", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)
df['Cleaned Price'] = pd.to_numeric(df['Cleaned Price'], errors='coerce')
df['Store'] = 'Naivas'

# ---------- Extract Weight (kept for internal use but no longer shown) ----------
def extract_weight(product):
    match = re.search(r'(\d+(?:\.\d+)?\s?(?:g|kg|ml|l|L))', product, re.IGNORECASE)
    return match.group(1).replace(" ", "").upper() if match else "Other"

df['Weight'] = df['Product'].apply(extract_weight)

# ---------- Categorize Products ----------
product_types = [
    "Cooking Oil", "Vegetable Oil", "Sunflower Oil", "Corn Oil", "Maize Flour", "Wheat Flour",
    "Sugar", "Salt", "Rice", "Toilet Paper", "Tissue", "Milk", "Water", "Bread", "Detergent",
    "Soap", "Sanitary Pads", "Batteries", "Tea", "Coffee", "Juice", "Soda", "Diapers", "Biscuits", 
    "Spaghetti", "Cereal", "Butter", "Cheese", "Toothpaste", "Shampoo", "Yogurt", "Cake",
    "Snacks", "Sauce", "Margarine", "Spices"
]

brand_to_type_map = {
    "Clovers": "Sauce", "Blue Band": "Margarine", "Ajab": "Wheat Flour",
    "Rina": "Cooking Oil", "Golden Fry": "Cooking Oil", "Kimbo": "Cooking Fat",
    "Elianto": "Sunflower Oil", "Kabras": "Sugar", "Soko": "Maize Flour", 
    "Omo": "Detergent", "Unga": "Maize Flour", "Royco": "Spices", "Fresh Fri": "Vegetable Oil",
    "Pembe": "Wheat Flour", "Daawat": "Rice", "Brookside": "Milk", "Velvex": "Toilet Paper",
    "Always": "Sanitary Pads", "Ariel": "Detergent", "Hanan": "Tissue", "Colgate": "Toothpaste",
    "Fanta": "Soda", "Coca Cola": "Soda", "Del Monte": "Juice", "Golden Penny": "Spaghetti"
}

def classify_type(product):
    product_lower = product.lower()
    for brand, category in brand_to_type_map.items():
        if brand.lower() in product_lower:
            return category
    for t in product_types:
        if t.lower() in product_lower:
            return t
    return 'Other'

df['Type'] = df['Product'].apply(classify_type)

# ---------- Simulate Carrefour & Quickmart ----------
def create_mock_store(df, store_name):
    mock = df.copy()
    variation = np.random.uniform(0.95, 1.15, len(mock))
    mock['Cleaned Price'] = (mock['Cleaned Price'] * variation).round(2)
    mock['Price'] = "KES " + mock['Cleaned Price'].apply(lambda x: f"{x:,.2f}")
    mock['Store'] = store_name
    return mock

df_carrefour = create_mock_store(df, 'Carrefour')
df_quickmart = create_mock_store(df, 'Quickmart')

df = pd.concat([df, df_carrefour, df_quickmart], ignore_index=True)

# ---------- Filters ----------
type_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df['Type'].unique()), 
    description="Category:",
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)
store_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df['Store'].unique()), 
    description="Retailer:",
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

output_table = widgets.Output()
bar_widget = widgets.Output()
line_widget = widgets.Output()
kpi_widget = widgets.HBox()

# ---------- KPI Cards ----------
def render_kpis(filtered):
    total = len(filtered)
    avg = filtered['Cleaned Price'].mean()
    min_ = filtered['Cleaned Price'].min()
    max_ = filtered['Cleaned Price'].max()
    
    kpi_widget.children = [
        HTML(f"""<div class="kpi-card" style="background: linear-gradient(135deg, #3498db, #2c3e50);">
            <div class="kpi-title">Total Products</div>
            <div class="kpi-value">{total}</div></div>"""),
        HTML(f"""<div class="kpi-card" style="background: linear-gradient(135deg, #2ecc71, #27ae60);">
            <div class="kpi-title">Avg Price</div>
            <div class="kpi-value">KES {avg:,.2f}</div></div>"""),
        HTML(f"""<div class="kpi-card" style="background: linear-gradient(135deg, #e74c3c, #c0392b);">
            <div class="kpi-title">Min Price</div>
            <div class="kpi-value">KES {min_:,.2f}</div></div>"""),
        HTML(f"""<div class="kpi-card" style="background: linear-gradient(135deg, #f39c12, #d35400);">
            <div class="kpi-title">Max Price</div>
            <div class="kpi-value">KES {max_:,.2f}</div></div>""")
    ]

# ---------- Bar Chart ----------
def render_bar(filtered):
    bar_widget.clear_output()
    with bar_widget:
        avg_by_type = filtered.groupby('Type')['Cleaned Price'].mean().sort_values()
        plt.figure(figsize=(10,5))
        avg_by_type.plot(kind='barh')
        plt.title("Average Price by Category")
        plt.xlabel("KES")
        plt.tight_layout()
        plt.show()

# ---------- Line Chart ----------
def render_line(filtered):
    line_widget.clear_output()
    with line_widget:
        grouped = filtered.groupby(['Type', 'Store'])['Cleaned Price'].mean().unstack()
        grouped.plot(marker='o')
        plt.title("Avg Price by Category per Retailer")
        plt.ylabel("KES")
        plt.tight_layout()
        plt.show()

# ---------- Product Table (Updated) ----------
def display_table():
    output_table.clear_output()
    with output_table:
        filtered = df.copy()
        if type_dropdown.value != 'All':
            filtered = filtered[filtered['Type'] == type_dropdown.value]
        if store_dropdown.value != 'All':
            filtered = filtered[filtered['Store'] == store_dropdown.value]

        # Add Predicted Price (5% higher)
        filtered['Predicted Price'] = (filtered['Cleaned Price'] * 1.05).round(2)

        render_kpis(filtered)
        render_bar(filtered)
        render_line(filtered)

        # Build HTML rows
        rows = ""
        for _, row in filtered.iterrows():
            rows += f"""
                <tr>
                    <td>{row['Product']}</td>
                    <td>{row['Price']}</td>
                    <td>{row['Type']}</td>
                    <td>{row['Store']}</td>
                    <td>KES {row['Predicted Price']:,.2f}</td>
                </tr>"""

        # Render HTML Table
        html = f"""
        <div class="table-container">
            <table>
                <thead>
                    <tr>
                        <th>Item</th>
                        <th>Price</th>
                        <th>Category</th>
                        <th>Retailer</th>
                        <th>Predicted Price</th>
                    </tr>
                </thead>
                <tbody>{rows}</tbody>
            </table>
        </div>
        """
        display(HTML(html))

# ---------- On Filter Change ----------
def on_filter_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        display_table()

for dropdown in [type_dropdown, store_dropdown]:
    dropdown.observe(on_filter_change)

# ---------- Layout ----------
dashboard = widgets.VBox([
    HTML("""
<style>
    .kpi-card {
        padding: 15px; border-radius: 8px; margin: 0 10px; min-width: 150px;
        text-align: center; box-shadow: 0 2px 4px rgba(0,0,0,0.1); color: white;
    }
    .kpi-title { font-size: 14px; margin-bottom: 5px; font-weight: bold; }
    .kpi-value { font-size: 18px; font-weight: bold; }
    .table-container { overflow-x: auto; }
    table { width: 100%; border-collapse: collapse; }
    th {
        background: #3498db; color: white; padding: 10px; text-align: left;
    }
    td { padding: 8px 10px; border-bottom: 1px solid #ddd; }
    tr:hover { background-color: #f5f5f5; }
    .title-container {
        display: flex; align-items: center; gap: 15px; margin-bottom: 20px;
    }
    .title-container img {
        height: 60px;
    }
    .title-container h2 {
        margin: 0; font-size: 28px;
    }
</style>
<div class="title-container">
    <img src="static/beiwatch_logo.jpeg" alt="BeiWatch Logo">
    <h2>BeiWatch – Kenya Cost of Living Dashboard</h2>
</div>
""")


,
    kpi_widget,
    widgets.HBox([bar_widget, line_widget]),
    widgets.HBox([type_dropdown, store_dropdown]),
    output_table
])

# ---------- Launch Dashboard ----------
display(dashboard)
display_table()
